# **Evaluación y Comparación de Resultados**

Evaluación del modelo GNN entrenado y comparación con heurísticas tradicionales.

In [ ]:
%run librerias.ipynb

In [ ]:
%run gnn_model.ipynb

In [ ]:
%run training.ipynb

## **1. FUNCIÓN DE EVALUACIÓN**

In [ ]:
def evaluate(model, data):
    model.eval()

    # Greedy baseline (orden natural)
    natural_order = list(range(data.num_nodes))
    t0 = time.time()
    colors_greedy = greedy_coloring_gnn(
        data.edge_index, data.num_nodes, natural_order
    )
    t1 = time.time()

    # GNN-guided greedy
    with torch.no_grad():
        scores = model(data.x, data.edge_index)
        ordering = torch.argsort(scores, descending=True).tolist()

    t2 = time.time()
    colors_gnn = greedy_coloring_gnn(
        data.edge_index, data.num_nodes, ordering
    )
    t3 = time.time()

    return {
        "greedy_colors": colors_greedy,
        "greedy_time": t1 - t0,
        "gnn_colors": colors_gnn,
        "gnn_time": (t3 - t2)
    }

## **2. EVALUACIÓN DEL MODELO**

In [ ]:
print("Evaluando modelo...")
print("="*60)
results = evaluate(model, data)

print("\nRESULTADOS DE EVALUACIÓN")
print("="*60)
for k, v in results.items():
    print(f"{k}: {v}")
print("="*60)

## **3. COMPARACIÓN CON HEURÍSTICAS**

In [ ]:
mejora_colores = results["greedy_colors"] - results["gnn_colors"]
mejora_porcentual = (mejora_colores / results["greedy_colors"]) * 100

print("\nANÁLISIS COMPARATIVO")
print("="*60)
print(f"Colores Greedy Baseline: {results['greedy_colors']}")
print(f"Colores GNN: {results['gnn_colors']}")
print(f"Mejora absoluta: {mejora_colores}")
print(f"Mejora porcentual: {mejora_porcentual:.2f}%")
print(f"\nTiempo Greedy: {results['greedy_time']:.6f}s")
print(f"Tiempo GNN: {results['gnn_time']:.6f}s")
print("="*60)

## **4. RESUMEN FINAL**

In [ ]:
print("\n" + "#"*60)
print("# RESUMEN FINAL DEL PROYECTO")
print("#"*60)
print(f"\nGrafo evaluado: {data.num_nodes} nodos, {data.edge_index.shape[1]} aristas")
print(f"\nMétodo Greedy Baseline: {results['greedy_colors']} colores")
print(f"Método GNN: {results['gnn_colors']} colores")

if mejora_colores > 0:
    print(f"\n✓ El modelo GNN logró una mejora de {mejora_colores} colores ({mejora_porcentual:.2f}%)")
elif mejora_colores == 0:
    print(f"\n= Ambos métodos obtuvieron el mismo número de colores")
else:
    print(f"\n✗ El modelo GNN usó {abs(mejora_colores)} colores más que el baseline")

print("\n" + "#"*60)